In [202]:
# peep peeep
!pip install pandas -q
# Load bar (i forgot to use it of course)
!pip install tqdm
# the engine behind the geo reverse process
!pip install geopy

In [203]:
# Importing our badbois
import pandas as pd
from tqdm import tqdm
from geopy.geocoders import Nominatim

In [204]:
known_geos = pd.read_csv('Known_UserDistrict_Geodata.csv') #LOAD DAT DATA

In [205]:
geolocator = Nominatim(user_agent="geoapiExercises") # This is a class object that can give us the text-based location data from our lat/lon.

In [206]:
ID_filter = list(known_geos.Userinfo_Key.unique()) # We need a list of unique user ID's so we don't get more than one from each

In [207]:
# I need to find the most frequent value for lat and lon, as this would be where we'd expect to find our users. Some users may have more than one, and so we get a list of 2 or more
known_geos.loc[known_geos['Userinfo_Key'] == 474737].Lat.mode()[0] #Checking if I can select a value out of the list that appeared without [0]

19.2283004

In [208]:
#I wanna create a loop to do the annoying work, so we loop our unique ID's through and using the code before, we fill 2 lists for the next part.
lat_list = [] # We generate 2 empty lists to fill with the lat/lon data for each user. The indexes of the lists will match up for the imputation later on.
lon_list = []
for ID in tqdm(ID_filter): #We need the filtered list of unique ID's for the users we found
    lat_temp = known_geos.loc[known_geos['Userinfo_Key'] == ID].Lat.mode()[0] # We find the most frequent Lat data for each user. If they got more than one, we just pick the first on the list.
    lon_temp = known_geos.loc[known_geos['Userinfo_Key'] == ID].Lon.mode()[0] # We do the same for the Lon data.
    lat_list.append(lat_temp) # We append the data to the empty lists we created earlier.
    lon_list.append(lon_temp)

100%|██████████| 1619/1619 [00:04<00:00, 382.74it/s]


In [209]:
print(str(lat_list[0])+ ','+ str(lon_list[0])) # The model takes the lat/lon input as a string with a comma separator

19.2283004,82.552138


In [210]:
# Let's try out this badboi on the first user (the list indexes match with the ID_filter we passed for the loop, which makes assignment easier later)
location = geolocator.reverse(str(lat_list[0])+ ','+ str(lon_list[0]))

In [211]:
print(location) # Woooo, we get some goodies. One step closer to finding those users

NH26, Nabarangapur, Nabarangapur P.S, Nabarangapur, Odisha, 764059, India


In [212]:
# There is a way to get a dictionary of the elements created from that Lat/Lon data. For the purpose of recreating the df_users we are interested in the state_district, state and country values 
address = location.raw['address']
print(address)

{'road': 'NH26', 'town': 'Nabarangapur', 'county': 'Nabarangapur P.S', 'state_district': 'Nabarangapur', 'state': 'Odisha', 'ISO3166-2-lvl4': 'IN-OR', 'postcode': '764059', 'country': 'India', 'country_code': 'in'}


In [213]:
district = location.raw['address'].get('state_district','') # We're only interested in the districts though

In [214]:
district # Seems to work

'Nabarangapur'

In [215]:
user_district_df = pd.DataFrame(ID_filter, columns=['Userinfo_Key']) # Let's recreate a dataframe from our ID_list

In [216]:
user_district_df.head(5)

,Userinfo_Key
0,474737
1,474986
2,476738
3,481106
4,481511


In [217]:
# We create a list, and run each ID through our geolocator to get the districts. These will match up indexwise with our dataframe
found_you = [] # Sorry about the not so creative names, I'll do better next time, promise :) 
for lat, lon in tqdm(zip(lat_list, lon_list)):
    location = geolocator.reverse(str(lat)+ ','+ str(lon))
    district = location.raw['address'].get('state_district','')
    found_you.append(district)

# There's 1619 unique UserKeys, so it performs 1619 iterations (1619it)?

1619it [13:59,  1.93it/s]


In [218]:
found_you_df = pd.DataFrame(found_you,columns = ['UserDistrict'])

In [219]:
# And now we just concatenate the dataframes;

found_users_df = pd.concat([user_district_df, found_you_df],axis=1)
found_users_df.head(10)

,Userinfo_Key,UserDistrict
0,474737,Nabarangapur
1,474986,Jammu District
2,476738,Faridabad
3,481106,Kanpur Nagar
4,481511,Pauri Garhwal
5,482082,አዲስ አበባ / Addis Ababa
6,483043,Cachar
7,484768,Raebareli
8,486277,Bastar
9,487821,Kamrup Metropolitan


In [264]:
### After this point, I am just cleaning the outputs of the location scrape. This is just to show how we did it, and not something to just be copy/pasted necessarily.




# There might be a smarter way of implementing NLP to perform the majority of the final cleaning/translations from the Google scrape, but we didn't have time to properly look into those solutions at the time of writing this.


In [265]:
found_users_df['UserDistrict'].unique()

array(['Nabarangapur', 'Jammu District', 'Faridabad', 'Kanpur Nagar',
       'Pauri Garhwal', ' Addis Ababa', 'Cachar', 'Raebareli', 'Bastar',
       'Kamrup Metropolitan', 'Pathanamthitta', 'Muzaffarpur', 'Neemuch',
       'Bale', 'Zone 1', 'Patan', 'Chitrakoot', 'Samastipur',
       'Dahod District', 'Vidisha', 'Singrauli', 'Tapi District',
       'Anand District', 'Sikar District', 'Khargone',
       'Bhilwara District', 'Uttar Dinajpur', 'Gautam Buddha Nagar',
       'Saharanpur', 'Dehradun', 'Murshidabad', 'Gonda', 'South Delhi',
       'Haveri district', 'Kolkata', 'Sahibzada Ajit Singh Nagar',
       'South 24 Parganas', 'North Gondar', 'Jajapur', 'Hugli',
       'Srinagar District', 'Darjeeling District', 'Purbi Singhbhum',
       'North Shewa (Oromia Region)', 'Nagaur District', 'Kheda District',
       'Moga', 'Amritsar', 'Vizianagaram', 'Ghazipur',
       'Jabalpur District', 'Varanasi', 'Shamli', 'Mahoba', 'Mau',
       'Rajkot', 'Hapur', 'Moradabad', 'Bareilly District',
 

In [221]:
#Since we had some weird names in our list, we will try and clean it up abit. The .split('/') function will split strings by a given separator
found_you[5].split('/')[-1] #If we add the minus one, we should always get the last (and hopefully correct district name)

' Addis Ababa'

In [222]:
found_you[2].split('/')[-1] #It also works for the ones without a / separator

'Faridabad'

In [223]:
# Let's try to get these districts looking a bit nicer without the non-english names. We write a for loop with the above code;
really_found_you = []
for x in tqdm(range(len(found_you))):
    I_dont_know = found_you[x].split('/')[-1] # adding the bits of code i tested above
    really_found_you.append(I_dont_know) #Append to our empty list. Indexes should match up

100%|██████████| 1619/1619 [00:00<00:00, 16808.11it/s]


In [224]:
really_found_you[39] # we still got some weird names that aren't english. After a quick look through them, the "patterns" in lack of better words seem to be repeating the same 2 again.
# So we let's first replace those. 
#We got: 
# ሰሜን ጎንደር = North Gondar and 
# ሰሜን ወሎ = North Wello (no I don't read ethiopean, but Google is my friend)

'ሰሜን ጎንደር'

In [225]:
#Let's replace them
for i in range(len(really_found_you)):
  
    
    if really_found_you[i] == 'ሰሜን ጎንደር':
        really_found_you[i] = 'North Gondar'
  
    
    if really_found_you[i] == 'ሰሜን ወሎ':
        really_found_you[i] = 'North Wello'

In [226]:
really_found_you[39] #sucess!

'North Gondar'

In [227]:
# Let's recreate our DF again now with the corrected values;
found_you_df = pd.DataFrame(really_found_you,columns=['UserDistrict'])

In [228]:
found_users_df = pd.concat([user_district_df, found_you_df],axis=1)
found_users_df

,Userinfo_Key,UserDistrict
0,474737,Nabarangapur
1,474986,Jammu District
2,476738,Faridabad
3,481106,Kanpur Nagar
4,481511,Pauri Garhwal
...,...,...
1614,928390,North Shewa (Amhara Region)
1615,928853,North Gondar
1616,935121,North Shewa (Amhara Region)
1617,939284,North Shewa (Amhara Region)


In [229]:
# As it turns out we still have roughly 130 users with empty strings for districts which isn't useful. Most of these are from roughly the same areas based on the lat/lon data
# so for these I will simply find the districts through google and add them manually (yes, sometimes efforts have to be lowtech)
really_found_you[118]

''

In [230]:
#Making a function to give me the indexes of the missing states.
def find_indices(list_to_check, item_to_find):
    indices = []
    for idx, value in enumerate(list_to_check):
        if value == item_to_find:
            indices.append(idx)
    return indices

In [231]:
blank_list = find_indices(really_found_you,'') #Creating my list of indexes that have '' empty string for state_district.

In [232]:
# Just creating a printed out list of indexes and matching lat/long to find them manually (it was initially 134 users with empty strings)
for i in blank_list:
    print(f'Index {i} lat/lon: ' + str(lat_list[i]) +","+str(lon_list[i]))

Index 36 lat/lon: 22.5159759,88.3929637
Index 118 lat/lon: 23.8072333,86.4701313
Index 124 lat/lon: 22.4734067,88.3695949
Index 129 lat/lon: 22.7519828,88.3713696
Index 157 lat/lon: 22.4880621,88.3935816
Index 168 lat/lon: 18.4348198,73.8480931
Index 169 lat/lon: 18.6206324,73.7353715
Index 173 lat/lon: 30.7081854,76.7785551
Index 176 lat/lon: 22.8957172,88.427064
Index 198 lat/lon: 31.4205292,76.6445826
Index 248 lat/lon: 28.4776379,76.9074673
Index 323 lat/lon: 28.6740771,77.4664234
Index 352 lat/lon: 28.8531986,77.5875557
Index 358 lat/lon: 6.60032,38.4242443
Index 366 lat/lon: 20.41981604,80.85105434
Index 387 lat/lon: 8.5367352,39.2621208
Index 398 lat/lon: 8.577749,39.2895764
Index 399 lat/lon: 22.0715715,81.6834564
Index 406 lat/lon: 23.2992656,81.3979299
Index 413 lat/lon: 18.7900281,80.8119131
Index 443 lat/lon: 11.594148,37.3822772
Index 472 lat/lon: 7.0323651,38.4781653
Index 477 lat/lon: 6.7869172,47.4153237
Index 509 lat/lon: 7.0419751,38.4703338
Index 522 lat/lon: 9.59780

In [233]:
indexes = [36, 124, 129, 157 ,176,1043,1159] #Doing all the replacements after manually finding them
replacements = list(len(indexes) * ('Kolkata',))    

for (index, replacement) in zip(indexes, replacements):
    really_found_you[index] = replacement
    

indexes = [118]
replacements = list(len(indexes) * ('Dhanbad',))        

for (index, replacement) in zip(indexes, replacements):
    really_found_you[index] = replacement
    

indexes = [168, 169,1423]
replacements = list(len(indexes) * ('Pune',))

for (index, replacement) in zip(indexes, replacements):
    really_found_you[index] = replacement


indexes = [173]
replacements = list(len(indexes) * ('Chandigarh',))   

for (index, replacement) in zip(indexes, replacements):
    really_found_you[index] = replacement
    

indexes = [198]
replacements = list(len(indexes) * ('Bilaspur',))

for (index, replacement) in zip(indexes, replacements):
    really_found_you[index] = replacement
    
indexes = [248]
replacements = list(len(indexes) * ('Gurgaon',))    

for (index, replacement) in zip(indexes, replacements):
    really_found_you[index] = replacement
    

indexes = [323]
replacements = list(len(indexes) * ('Delhi',))    

for (index, replacement) in zip(indexes, replacements):
    really_found_you[index] = replacement
    

indexes = [352]
replacements = list(len(indexes) * ('Ghaziabad',))    

for (index, replacement) in zip(indexes, replacements):
    really_found_you[index] = replacement
    

indexes = [358, 472, 509, 580, 585, 620,834,849,956,1004,1012,1020,1063, 1071,1103,1187,1192,1252,1340,1402,1412]
replacements = list(len(indexes) * ('Sidama Region',))    

for (index, replacement) in zip(indexes, replacements):
    really_found_you[index] = replacement
    

indexes = [366]
replacements = list(len(indexes) * ('Rajnandgaon',))
    
for (index, replacement) in zip(indexes, replacements):
    really_found_you[index] = replacement
        

indexes = [387,398,579,607,647,678,686,754,755,827,864,1077,1092,1095,1121,1123,1125,1126,1127,1130,1173, 1177,1181,1209,1210,1211,1212,1214,1220,1224,1231,1246, 1360,1467,1536,1554,1575,1583,1585,1589,1595,1596,1597,1598,1599,1600,1601,1607]
replacements = list(len(indexes) * ('Oromia Region',))    
for (index, replacement) in zip(indexes, replacements):
    really_found_you[index] = replacement
    

indexes = [399]
replacements = list(len(indexes) * ('Mungeli',))    
for (index, replacement) in zip(indexes, replacements):
    really_found_you[index] = replacement
    

indexes = [406]
replacements = list(len(indexes) * ('Giridih',))    
for (index, replacement) in zip(indexes, replacements):
    really_found_you[index] = replacement
    

indexes = [413]
replacements = list(len(indexes) * ('Bijapur',))    
for (index, replacement) in zip(indexes, replacements):
    really_found_you[index] = replacement
    

indexes = [1338]
replacements = list(len(indexes) * ('Sarlahi',))    
for (index, replacement) in zip(indexes, replacements):
    really_found_you[index] = replacement
    
indexes = [1021]
replacements = list(len(indexes) * ('Sonipat',))    
for (index, replacement) in zip(indexes, replacements):
    really_found_you[index] = replacement
    

indexes = [443, 643, 786,984,1051,1179,1375,1511]
replacements = list(len(indexes) * ('Amhara Region',))    
for (index, replacement) in zip(indexes, replacements):
    really_found_you[index] = replacement
    

indexes = [522, 523, 524, 527, 553, 558, 562, 586,595,777,847,927,929,940,951,960,991, 1038, 1059,1105,1120, 1290, 1293, 1464,1470,1474]
replacements = list(len(indexes) * ('Dire Dawa Region',))    
for (index, replacement) in zip(indexes, replacements):
    really_found_you[index] = replacement
    

indexes = [477]
replacements = list(len(indexes) * ('Mudug Region',))    
for (index, replacement) in zip(indexes, replacements):
    really_found_you[index] = replacement
    

indexes = [644,1277]
replacements = list(len(indexes) * ('Somali Region',))    
for (index, replacement) in zip(indexes, replacements):
    really_found_you[index] = replacement
    

indexes = [669,1369]
replacements = list(len(indexes) * ('Mandera Triangle',))    
for (index, replacement) in zip(indexes, replacements):
    really_found_you[index] = replacement
        

indexes = [1364]
replacements = list(len(indexes) * ('Daedok Innopolis',))    
for (index, replacement) in zip(indexes, replacements):
    really_found_you[index] = replacement
        

indexes = [1387]
replacements = list(len(indexes) * ('Al Faw',))    
for (index, replacement) in zip(indexes, replacements):
    really_found_you[index] = replacement
        

indexes = [1478]
replacements = list(len(indexes) * ('Nairobi Area',))    
for (index, replacement) in zip(indexes, replacements):
    really_found_you[index] = replacement

indexes = [1268]
replacements = list(len(indexes) * ('Nyabihu',))    
for (index, replacement) in zip(indexes, replacements):
    really_found_you[index] = replacement

In [234]:
really_found_you[1170] #apparently the California Fire department has been looking at the app as well

'CAL Fire Southern Region'

In [235]:
blank_list = find_indices(really_found_you,'') #Performing another blank value check

In [236]:
blank_list #No more blank districts, yay.

[]

In [237]:
found_you_df = pd.DataFrame(really_found_you,columns=['UserDistrict']) # Creating the dataframe with final districts

In [238]:
found_users_df = pd.concat([user_district_df, found_you_df],axis=1)
found_users_df

,Userinfo_Key,UserDistrict
0,474737,Nabarangapur
1,474986,Jammu District
2,476738,Faridabad
3,481106,Kanpur Nagar
4,481511,Pauri Garhwal
...,...,...
1614,928390,North Shewa (Amhara Region)
1615,928853,North Gondar
1616,935121,North Shewa (Amhara Region)
1617,939284,North Shewa (Amhara Region)


In [239]:
# Now let's do the same steps for the country and state names;

In [240]:
# We create a list, and run each ID through our geolocator to get the countries. These will match up indexwise with our dataframe
found_your_country = []
for lat, lon in tqdm(zip(lat_list, lon_list)):
    location = geolocator.reverse(str(lat)+ ','+ str(lon))
    country = location.raw['address'].get('country','')
    found_your_country.append(country)

1619it [14:04,  1.92it/s]


In [241]:
# We create a list, and run each ID through our geolocator to get the states. These will match up indexwise with our dataframe
found_your_state = []
for lat, lon in tqdm(zip(lat_list, lon_list)):
    location = geolocator.reverse(str(lat)+ ','+ str(lon))
    state = location.raw['address'].get('state','')
    found_your_state.append(state)

1619it [13:29,  2.00it/s]


In [242]:
found_your_country.index('United States') #Was a bit confused about a US user, but mostly just needed to list up the 'weird' names
# ኢትዮጵያ = Ethiopia
# Soomaaliya الصومال = Somalia
# नेपाल = Nepal
# 대한민국 = South Korea
# السودان = Sudan
# বাংলাদেশ = Bangladesh

1170

In [243]:
country_df = pd.DataFrame(found_your_country,columns=['UserCountry'])

In [244]:
country_df['UserCountry'].unique() #I create a dataframe to get out the unique values

array(['India', 'ኢትዮጵያ', 'Soomaaliya الصومال', 'Kenya', 'United States',
       'Rwanda', 'नेपाल', '대한민국', 'السودان', 'বাংলাদেশ'], dtype=object)

In [245]:
#Let's replace them
for i in range(len(found_your_country)):
  
    
    if found_your_country[i] == 'Soomaaliya الصومال':
        found_your_country[i] = 'Somalia'
  
    
    if found_your_country[i] == 'ኢትዮጵያ':
        found_your_country[i] = 'Ethiopia'
    
    if found_your_country[i] == '대한민국':
        found_your_country[i] = 'South Korea'
        
    if found_your_country[i] == 'السودان':
        found_your_country[i] = 'Sudan'
    
    if found_your_country[i] == 'বাংলাদেশ':
        found_your_country[i] = 'Bangladesh'
    
    if found_your_country[i] == 'नेपाल':
        found_your_country[i] = 'Nepal'
        

In [246]:
country_df = pd.DataFrame(found_your_country,columns=['UserCountry']) #Let's remake the DF with our corrected values for another check before we concat

In [247]:
country_df['UserCountry'].unique() # Looks good

array(['India', 'Ethiopia', 'Somalia', 'Kenya', 'United States', 'Rwanda',
       'Nepal', 'South Korea', 'Sudan', 'Bangladesh'], dtype=object)

In [248]:
found_users_df = pd.concat([found_users_df, country_df],axis=1) # Let's put this next part of the dataframe together
found_users_df

,Userinfo_Key,UserDistrict,UserCountry
0,474737,Nabarangapur,India
1,474986,Jammu District,India
2,476738,Faridabad,India
3,481106,Kanpur Nagar,India
4,481511,Pauri Garhwal,India
...,...,...,...
1614,928390,North Shewa (Amhara Region),Ethiopia
1615,928853,North Gondar,Ethiopia
1616,935121,North Shewa (Amhara Region),Ethiopia
1617,939284,North Shewa (Amhara Region),Ethiopia


In [249]:
# That's country names done, let's take a look at your state names

In [250]:
state_df = pd.DataFrame(found_your_state,columns=['UserState']) #Let's do the same for the State data. This time we will have to repeat some of the cleaning steps from earlier

In [251]:
state_df['UserState'].unique()

array(['Odisha', 'Jammu and Kashmir', 'Haryana', 'Uttar Pradesh',
       'Uttarakhand', 'አዲስ አበባ', 'Assam', 'Chhattisgarh', 'Kerala',
       'Bihar', 'Madhya Pradesh', 'ኦሮሚያ ክልል / Oromia', 'አፋር ክልል / Afar',
       'Gujarat', 'Rajasthan', 'West Bengal', 'Delhi', 'Karnataka',
       'Punjab', 'አማራ ክልል / Amhara', 'Jharkhand', 'Andhra Pradesh',
       'Tamil Nadu', 'Mizoram', 'Nagaland', 'Puducherry', 'Telangana',
       'Himachal Pradesh', 'Maharashtra', '', 'ሶማሌ ክልል / Somali',
       "የደቡብ ብሔር ብሔረሰቦችና ሕዝቦች ክልል / Southern Nations, Nationalities, and Peoples' Region",
       'Arunachal Pradesh', 'ሲዳማ ክልል / Sidama', 'ጋምቤላ ሕዝቦች ክልል / Gambela',
       'Mudug مدج', 'ሐረሪ ሕዝብ ክልል / Harar', 'ድሬዳዋ', 'ትግራይ',
       'ቤንሻንጉል ጉሙዝ ክልል / Benishangul-Gumuz', 'Gedo جدو', 'Mandera',
       'Meghalaya', 'California', 'Iburengerazuba', 'القضارف', 'Manipur',
       'খুলনা বিভাগ', 'Nairobi'], dtype=object)

In [252]:
found_states = [] # Doing the splitsies first so we mostly get english text;
for x in tqdm(range(len(found_your_state))):
    I_dont_know = found_your_state[x].split('/')[-1] # adding the bits of code i tested earlier
    found_states.append(I_dont_know)

100%|██████████| 1619/1619 [00:00<00:00, 566353.48it/s]


In [253]:
found_states_df = pd.DataFrame(found_states,columns=['UserState'])
found_states_df['UserState'].unique()

array(['Odisha', 'Jammu and Kashmir', 'Haryana', 'Uttar Pradesh',
       'Uttarakhand', 'አዲስ አበባ', 'Assam', 'Chhattisgarh', 'Kerala',
       'Bihar', 'Madhya Pradesh', ' Oromia', ' Afar', 'Gujarat',
       'Rajasthan', 'West Bengal', 'Delhi', 'Karnataka', 'Punjab',
       ' Amhara', 'Jharkhand', 'Andhra Pradesh', 'Tamil Nadu', 'Mizoram',
       'Nagaland', 'Puducherry', 'Telangana', 'Himachal Pradesh',
       'Maharashtra', '', ' Somali',
       " Southern Nations, Nationalities, and Peoples' Region",
       'Arunachal Pradesh', ' Sidama', ' Gambela', 'Mudug مدج', ' Harar',
       'ድሬዳዋ', 'ትግራይ', ' Benishangul-Gumuz', 'Gedo جدو', 'Mandera',
       'Meghalaya', 'California', 'Iburengerazuba', 'القضارف', 'Manipur',
       'খুলনা বিভাগ', 'Nairobi'], dtype=object)

In [254]:
# Replacing for english names; 
# አዲስ አበባ = Addis Ababa
# Mudug مدج = Mudug
# ድሬዳዋ = Dire Dawa
# ትግራይ = Tigray
# Gedo جدو = Gedo
# القضارف = Al Qadarif
# খুলনা বিভাগ = Khulna Division

In [255]:
#Let's replace/translate/clean up the ones we already know;
for i in range(len(found_states)):
  
    
    if found_states[i] == 'አዲስ አበባ':
        found_states[i] = 'Addis Ababa'
  
    
    if found_states[i] == 'Mudug مدج':
        found_states[i] = 'Mudug'
    
    if found_states[i] == 'ድሬዳዋ':
        found_states[i] = 'Dire Dawa'
        
    if found_states[i] == 'ትግራይ':
        found_states[i] = 'Tigray'
    
    if found_states[i] == 'Gedo جدو':
        found_states[i] = 'Gedo'
    
    if found_states[i] == 'القضارف':
        found_states[i] = 'Al Qadarif'
    
    if found_states[i] == 'খুলনা বিভাগ':
        found_states[i] = 'Khulna Division'

In [256]:
blank_list = find_indices(found_states,'') #Performing another blank value check
blank_list #Only a few blank states, which is manageable to find.

[173,
 461,
 473,
 508,
 512,
 535,
 600,
 740,
 774,
 783,
 840,
 913,
 955,
 967,
 1073,
 1124,
 1166,
 1338,
 1364,
 1557]

In [257]:
for i in blank_list: #Creating the list of blank indexes and their corresponding lat/lon to find the country. They seem to be in 3-4 general areas, so that makes it a lot easier.
    print(f'Index {i} lat/lon: ' + str(lat_list[i]) +","+str(lon_list[i]))

Index 173 lat/lon: 30.7081854,76.7785551
Index 461 lat/lon: 6.9995648,35.5902338
Index 473 lat/lon: 7.0141415,37.2720475
Index 508 lat/lon: 7.3015715,36.2311204
Index 512 lat/lon: 7.2007231,35.414186
Index 535 lat/lon: 7.7484153,35.4762193
Index 600 lat/lon: 7.1179448,36.1462237
Index 740 lat/lon: 7.2750214,36.2387556
Index 774 lat/lon: 7.1172462,36.4697523
Index 783 lat/lon: 6.8761637,35.5208036
Index 840 lat/lon: 7.1487538,35.8112923
Index 913 lat/lon: 7.1448886,35.800893
Index 955 lat/lon: 6.9582235,35.5574072
Index 967 lat/lon: 7.0608879,35.4648514
Index 1073 lat/lon: 7.176276,36.7939001
Index 1124 lat/lon: 7.1177607,36.470417
Index 1166 lat/lon: 7.682416667,35.26707667
Index 1338 lat/lon: 26.8681696,85.5517178
Index 1364 lat/lon: 36.3741546,127.3654559
Index 1557 lat/lon: 6.9545739,35.5519171


In [258]:
indexes = [173] # Replacing the empty strings with the state names
replacements = list(len(indexes) * ('Himachal Pradesh',))    
for (index, replacement) in zip(indexes, replacements):
    found_states[index] = replacement
    
indexes = [461,473,508,512,535,600,740,774,783,840,913,955,967,1073,1124,1166,1557,1557]
replacements = list(len(indexes) * ("Southern Nations, Nationalities, and People's Region",))    
for (index, replacement) in zip(indexes, replacements):
    found_states[index] = replacement
    
indexes = [1338]
replacements = list(len(indexes) * ('Madhesh Pradesh',))    
for (index, replacement) in zip(indexes, replacements):
    found_states[index] = replacement
    
indexes = [1364]
replacements = list(len(indexes) * ('Daejon',))    
for (index, replacement) in zip(indexes, replacements):
    found_states[index] = replacement

In [259]:
found_states.index('') #Checking for any more empty strings. And there is no more empty strings, yaay

ValueError: '' is not in list

In [261]:
 #Assemble the final dataframe. (Don't be a dummy like me and run this twice accidentally)
found_states_df = pd.DataFrame(found_states,columns=['UserState'])

In [ ]:
found_users_df1 = pd.concat([found_users_df,found_states_df],axis=1)

In [ ]:
# FINAL NOTES: With this code we simply assume that the most pinged GPS coordinates from the user, is their true locations.
# If any other true data says otherwise (such as the Userinfo_State or other location data), this assumption is broken and these 'imputations' could very well be wrong
# If no other data states the true location for the missing data, the assumptions holds ground and these 'imputations' are the best estimator for the true values.

In [268]:
found_users_df1.loc[found_users_df1['UserCountry'] == 'United States']

,Userinfo_Key,UserDistrict,UserCountry,UserState,UserState,UserState
1170,772268,CAL Fire Southern Region,United States,California,California,California


In [150]:
#found_users_df.to_csv('Found_users_location_data.csv',index=False) # Out of habit, we saved it to a CSV file in order to implement it in further work. 